<a href="https://colab.research.google.com/github/lucasHashi/Quarentena-dados-Alura/blob/master/Desafio%20Final/Minhas_analises_explicadas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
for chave in dados_categorizados:
  dados_atuais = dados_categorizados[chave]
  resultado_atual = avaliar_modelos(dados_atuais, modelos=['floresta'])
  print('\n------- {} notas e {} redacao -------'.format(chave.split('_')[0], chave.split('_')[1]))

  for modelo in resultado_atual:
    erro = resultado_atual[modelo]['erro']
    print('Modelo: {}\n Erro: {}'.format(modelo, erro))


------- 3 notas e True redacao -------
Modelo: floresta
 Erro: 300.6858060666357

------- 2 notas e True redacao -------
Modelo: floresta
 Erro: 300.10668035717634

------- 1 notas e True redacao -------
Modelo: floresta
 Erro: 300.8191028010589

------- 0 notas e True redacao -------
Modelo: floresta
 Erro: 300.7503064886648

------- 3 notas e False redacao -------
Modelo: floresta
 Erro: 300.90863316330564

------- 2 notas e False redacao -------
Modelo: floresta
 Erro: 301.108898113732

------- 1 notas e False redacao -------
Modelo: floresta
 Erro: 301.08919927886853

------- 0 notas e False redacao -------
Modelo: floresta
 Erro: 300.7786254515662


# Desafio final da Quarentena Dados da Alura

## Resumo

A análise começou com os **modelos**:
*   Regressão linear
*   SVR
*   Floresta aleatória
*   SGDRegressor

Mas tivemos duas baixas ao longo das análises:
*   SVR, pelo tempo de processamento e não demonstrar grande melhoria em relação a Regressão linear
*   SGDRegressor, pelo erro medio quadrático extremamente alto

Das **abordagens utilizadas**.

Método 1: Não separar ou limpar dados, utilizar a tabela inteira e rodas os modelos.

Método 2: Separar os alunos por categorias, em relação a quantas provas tiveram nota > 300 e pela nota de redação > 0

## Codigos base

In [0]:
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.metrics import mean_squared_error

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.linear_model import SGDRegressor


URI_TREINO = "https://github.com/tgcsantos/quaretenadados/blob/master/DADOS_TREINO.csv?raw=true"
URI_TESTE = "https://github.com/tgcsantos/quaretenadados/raw/master/DADOS_TESTE.csv"
URI_DESAFIOQT = "https://github.com/tgcsantos/quaretenadados/raw/master/DESAFIOQT.csv"

dados_treino = pd.read_csv(URI_TREINO)
dados_teste = pd.read_csv(URI_TESTE)
dados_desafioqt = pd.read_csv(URI_DESAFIOQT)

erro_treino = "Erro ao carregar dados de treino"
erro_teste = "Erro ao carregar dados de teste"
erro_desafioqt = "Erro ao carregar dados de submissão"

assert dados_treino.shape == (150000, 5), erro_treino
assert dados_teste.shape == (20000, 5), erro_teste
assert dados_desafioqt.shape == (10000, 5), erro_desafioqt

In [0]:
coluna_label = 'NU_NOTA_LC'
coluna_features = ['NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_MT', 'NU_NOTA_REDACAO']

X_treino = dados_treino[coluna_features].to_numpy()
Y_treino = dados_treino[coluna_label].to_numpy()
X_teste = dados_teste[coluna_features].to_numpy()
Y_teste = dados_teste[coluna_label].to_numpy()


In [35]:
#Exemplo de classificação com Dummy
from sklearn.dummy import DummyRegressor
from sklearn.metrics import mean_squared_error

modelo_dummy = DummyRegressor()
modelo_dummy.fit(X_treino, Y_treino)
dummy_predicoes = modelo_dummy.predict(X_teste)

avaliacao_dummy = mean_squared_error(Y_teste, dummy_predicoes)

print(f"Minha avaliação nos dados de teste foi de {avaliacao_dummy}")

Minha avaliação nos dados de teste foi de 5219.286870186777


# Inicio minhas análises

## Função de avaliação dos modelos
**Entrada:**
*   dados_treino - DataFrame de treino com todas as colunas
*   modelos - List com os modelos que devem ser testados. Default = ['linear', 'floresta', 'SVR', 'SGD'] (todos os modelos)

**Retorno:**
*   Dict - predições dos modelos listados





In [0]:
def avaliar_modelos(dados_treino, modelos = ['linear', 'floresta']):
  # Divisao dos dados para treinos e teste
  coluna_label = 'NU_NOTA_LC'
  coluna_features = ['NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_MT', 'NU_NOTA_REDACAO']

  X_treino = dados_treino_todas_notas[coluna_features].to_numpy()
  Y_treino = dados_treino_todas_notas[coluna_label].to_numpy()
  X_teste = dados_treino_todas_notas[coluna_features].to_numpy()
  Y_teste = dados_treino_todas_notas[coluna_label].to_numpy()

  dict_predicoes = {}

  # Para cada modelo escolhido
  while(modelos):
    nome_modelo = modelos.pop()
    if(nome_modelo == 'linear'):
      modelo = LinearRegression()
    elif(nome_modelo == 'SVR'):
      #modelo = SVR()
      modelo = LinearRegression()
    elif(nome_modelo == 'floresta'):
      modelo = RandomForestRegressor()
    
    # Avaliar e guardar:
    #   Predicoes para os dados de teste
    #   Respostas certas para os dados de teste
    #   Erro sobre os dados de teste
    dict_predicoes[nome_modelo] = {}
    modelo.fit(X_treino, Y_treino)
    predicoes = modelo.predict(X_teste)
    dict_predicoes[nome_modelo]['predicoes'] = predicoes
    dict_predicoes[nome_modelo]['corretas'] = Y_teste

    avaliacao = mean_squared_error(Y_teste, predicoes)

    dict_predicoes[nome_modelo]['erro'] = avaliacao
  
  return dict_predicoes

## Método de exploração nivel 1
Análise da base inteira.

Avaliar modelos sem qualquer divisão.

**O modelo SVR foi rodado um vez (em 50 minutos) e o resultado foi algo como 20 a menos do que a Regressão Linear.** Por isso foi tirado da análise.

In [24]:
avaliar_modelos(dados_treino)

Modelo: Regressao linear
 Erro: 2119.362708047893
SVR é pesado né :(
Modelo: Floresta aleatoria
 Erro: 300.4791777046204
Modelo: SGD Regressor
 Erro: 7.802905819839274e+26


In [134]:
modelo_atual = 'floresta'
print('Modelo utilizado: {}'.format(modelo_atual))

print('\n------- Todos os dados -------')
resultado = avaliar_modelos(dados_treino, modelos=[modelo_atual])
resultado = resultado[modelo_atual]

erro = resultado['erro']

print('Minha avaliação PARCIAL nos dados de teste foi de {}'.format(erro))

Modelo utilizado: floresta

------- Todos os dados -------
Minha avaliação PARCIAL nos dados de teste foi de 300.0860191414114


## Método de exploração nivel 2
Para este caso toma-se como hipótese que é a **quantidade** de matérias feitas que interfere na nota de portugues e não **quais** foram as matérias feitas

Ter nota ou não se refere a nota ser > 300 e para redação > 0

Análise da base divida pelas categorias:

*   Aluno com as 3 notas e com redação
*   Aluno com as 2 notas e com redação
*   Aluno com as 1 notas e com redação
*   Aluno com as 0 notas e com redação
*   Aluno com as 3 notas e sem redação
*   Aluno com as 2 notas e sem redação
*   Aluno com as 1 notas e sem redação
*   Aluno com as 0 notas e sem redação

Poderia ter separado em:

*   Aluno com 4 notas
*   Aluno com 3 notas
*   ...

Mas suponho que redação seja um fator importante para a nota em portugues

### Divisão das categorias

In [94]:
dados_treino_metodo1 = dados_treino.copy()

dados_treino_metodo1['CN_300'] = dados_treino_metodo1['NU_NOTA_CN'] > 300
dados_treino_metodo1['CH_300'] = dados_treino_metodo1['NU_NOTA_CH'] > 300
dados_treino_metodo1['MT_300'] = dados_treino_metodo1['NU_NOTA_MT'] > 300
dados_treino_metodo1['REDACAO_0'] = dados_treino_metodo1['NU_NOTA_REDACAO'] > 0

dados_treino_metodo1['CN_300'] = dados_treino_metodo1['CN_300'].apply(lambda x : 1 if(x) else 0)
dados_treino_metodo1['CH_300'] = dados_treino_metodo1['CH_300'].apply(lambda x : 1 if(x) else 0)
dados_treino_metodo1['MT_300'] = dados_treino_metodo1['MT_300'].apply(lambda x : 1 if(x) else 0)

dados_treino_metodo1['quant_notas'] = dados_treino_metodo1['CN_300'] + dados_treino_metodo1['CH_300'] + dados_treino_metodo1['MT_300']
dados_treino_metodo1

,NU_NOTA_CN,NU_NOTA_CH,NU_NOTA_LC,NU_NOTA_MT,NU_NOTA_REDACAO,CN_300,CH_300,MT_300,REDACAO_0,quant_notas
0,530.70,586.50,575.90,539.00,520.00,1,1,1,True,3
1,439.40,577.90,440.90,488.70,380.00,1,1,1,True,3
2,414.10,548.80,417.20,382.60,600.00,1,1,1,True,3
3,407.90,572.70,558.90,595.40,560.00,1,1,1,True,3
4,617.20,655.70,564.20,660.90,540.00,1,1,1,True,3
...,...,...,...,...,...,...,...,...,...,...
149995,587.80,644.10,621.40,714.40,840.00,1,1,1,True,3
149996,385.00,525.60,473.20,446.20,520.00,1,1,1,True,3
149997,471.00,558.70,582.40,542.90,460.00,1,1,1,True,3
149998,476.70,463.50,553.30,434.00,0.00,1,1,1,False,3


In [99]:
dados_categorizados = {}

for valida_redacao in [True, False]:
  for valida_notas in [3, 2, 1, 0]:
    chave = str(valida_notas) + '_' + str(valida_redacao)
    dados_atuais = dados_treino_metodo1[(dados_treino_metodo1['quant_notas'] == valida_notas) & (dados_treino_metodo1['REDACAO_0'] == valida_redacao)]

    dados_categorizados[chave] = dados_atuais

# Dados dos alunos que responderam duas das três provas e tiveram nota na redacao
dados_categorizados['2_True']


,NU_NOTA_CN,NU_NOTA_CH,NU_NOTA_LC,NU_NOTA_MT,NU_NOTA_REDACAO,CN_300,CH_300,MT_300,REDACAO_0,quant_notas
175,459.30,0.00,506.50,518.40,340.00,1,0,1,True,2
351,437.50,0.00,460.80,494.50,320.00,1,0,1,True,2
2874,534.10,0.00,433.30,502.30,580.00,1,0,1,True,2
3073,459.90,0.00,0.00,518.60,460.00,1,0,1,True,2
3250,474.50,0.00,0.00,489.40,440.00,1,0,1,True,2
...,...,...,...,...,...,...,...,...,...,...
147115,452.90,0.00,557.40,491.80,600.00,1,0,1,True,2
147494,441.90,0.00,0.00,469.60,400.00,1,0,1,True,2
148990,498.80,0.00,530.30,523.10,680.00,1,0,1,True,2
149104,544.80,0.00,0.00,376.60,320.00,1,0,1,True,2


### Análise dos resultados

**Pelo baixo erro do modelo Floresta Aleatória, esse será o modelo principal utilizado.**

Para analisar os resultados de todas as categorias juntas, vou juntar as predições em uma tabela única.

In [136]:
predicoes = []
corretas = []
modelo_atual = 'floresta'
print('Modelo utilizado: {}'.format(modelo_atual))

for chave in dados_categorizados:
#for chave in ['3_True', '2_True', '3_False']:
  dados_atuais = dados_categorizados[chave]
  print('\n------- {} notas e {} redacao -------'.format(chave.split('_')[0], chave.split('_')[1]))
  resultado_atual = avaliar_modelos(dados_atuais, modelos=[modelo_atual])
  resultado_atual = resultado_atual[modelo_atual]
  predicoes_atuais = resultado_atual['predicoes']
  corretas_atuais = resultado_atual['corretas']

  erro_atual = resultado_atual['erro']

  predicoes = np.concatenate((predicoes, predicoes_atuais), axis=0)
  corretas = np.concatenate((corretas, corretas_atuais), axis=0)

  print('Predicoes e respostas corretas adicionadas\n  Erro: {}'.format(erro_atual))

erro_total = mean_squared_error(corretas, predicoes)
print('Minha avaliação PARCIAL nos dados de teste foi de {}'.format(erro_total))

Modelo utilizado: floresta

------- 3 notas e True redacao -------
Predicoes e respostas corretas adicionadas
  Erro: 301.03388391107507

------- 2 notas e True redacao -------
Predicoes e respostas corretas adicionadas
  Erro: 300.22637310289707

------- 1 notas e True redacao -------
Predicoes e respostas corretas adicionadas
  Erro: 300.5638104110376

------- 0 notas e True redacao -------
Predicoes e respostas corretas adicionadas
  Erro: 301.5387647154144

------- 3 notas e False redacao -------
Predicoes e respostas corretas adicionadas
  Erro: 301.45568042121226

------- 2 notas e False redacao -------
Predicoes e respostas corretas adicionadas
  Erro: 300.8499236574015

------- 1 notas e False redacao -------
Predicoes e respostas corretas adicionadas
  Erro: 301.0125438907597

------- 0 notas e False redacao -------
Predicoes e respostas corretas adicionadas
  Erro: 301.3798703621445
Minha avaliação PARCIAL nos dados de teste foi de 301.0076063089929
